In [161]:
import pandas as pd

In [162]:
df = pd.read_csv("movie_metadata.csv")

In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   director_name              4939 non-null   object 
 2   num_critic_for_reviews     4993 non-null   float64
 3   duration                   5028 non-null   float64
 4   director_facebook_likes    4939 non-null   float64
 5   actor_3_facebook_likes     5020 non-null   float64
 6   actor_2_name               5030 non-null   object 
 7   actor_1_facebook_likes     5036 non-null   float64
 8   gross                      4159 non-null   float64
 9   genres                     5043 non-null   object 
 10  actor_1_name               5036 non-null   object 
 11  movie_title                5043 non-null   object 
 12  num_voted_users            5043 non-null   int64  
 13  cast_total_facebook_likes  5043 non-null   int64

In [164]:
df.duration.value_counts().sort_index().sum()

5028

Сколько пропущенных значений содержится в колонке duration?

In [246]:
df.shape[0]-df.duration.value_counts().sort_index().sum()

15

Замените пропущенные значения в колонке duration медианным значением этой колонки.

Какое среднее арифметическое по длительности фильма? Ответ дайте числом с плавающей запятой с округлением до 2 знаков после запятой включительно.


In [247]:
round((df
 .fillna(df.duration.median())
 .duration.describe()
 .get("mean")
),2)

107.19

Cоздайте колонку movie_duration_category, которая содержала бы 3 категории в зависимости от длительности фильма:

Категория "1. < 90", если фильм длится менее 90 минут;
Категория "2. 90-120", если фильм длится от 90 минут до 2 часов (включительно);
Категория "3. > 120", если фильм длится больше 2 часов.
Постройте сводную таблицу для фильмов с годом выпуска после 2000 (включительно), которая отражала бы количество фильмов:

В строках таблицы - год;
В столбцах таблицы - категория длительности фильма ("< 90", "90-120", "> 120");
Год фильма должен быть представлен в формате YYYY
Сколько фильмов с длительностью от 90 минут до 2 часов было выпущено в 2008 году?

In [251]:
df_1 = df.assign(movie_duration_category="1. < 90")
df_1.loc[df_1.duration >= 90, "movie_duration_category"] = "2. 90-120"
df_1.loc[df_1.duration > 120, "movie_duration_category"] = "3. > 120"
(df_1
 .loc[df_1.title_year==2008]
 .movie_duration_category.value_counts()
 ).loc["2. 90-120"]

160

Колонка plot_keywords содержит ключевые слова, характеризующие сюжет фильма. На основе данных этой колонки создайте колонку movie_plot_category, которая содержала бы 4 категории в зависимости от ключевых слов в колонке:

- Категория "love_and_death", если ключевые слова содержат и ключевое слово "love" и ключевое слово "death";
- Категория "love", если ключевые слова содержат ключевое слово "love";
- Категория "death", если ключевые слова содержат ключевое слово "death";
- Категория "other", если ключевые слова не соответствуют перечисленным выше условиям.
Обратите внимание на разделитель ключевых слов. Подумайте о том как правильно определять наличие ключевого слова в списке ключевых слов.

В колонке imdb_score указан пользовательский рейтинг фильма. Постройте таблицу, которая отражала бы средний рейтинг фильмов в зависимости от принадлежности к категории movie_plot_category.

Какой средний рейтинг у фильмов категории "love"? Ответ дайте числом с плавающей запятой с округлением до 2 знаков после запятой включительно.

In [275]:
def love_criteria(df):
    criteria = df[1]
    for clmn in df.columns:
        criteria = (criteria) | df[clmn]
    return criteria

new_df = df.assign(movie_plot_category = "other")
new_df.loc[
     (lambda x: x
      .plot_keywords.str.lower().str.split(r"[|\s]", expand=True)
      .isin(["love"])
      .pipe(love_criteria)
     ),
    "movie_plot_category" 
 ] = "love"
new_df.loc[
     (lambda x: x
      .plot_keywords.str.lower().str.split(r"[|\s]", expand=True)
      .isin(["death"])
      .pipe(love_criteria)
     ),
    "movie_plot_category" 
 ] = "death"
condition = (new_df
      .plot_keywords.str.lower().str.split(r"[|\s]", expand=True)
      .isin(["death"])
      .pipe(love_criteria)
     ) & (new_df
      .plot_keywords.str.lower().str.split(r"[|\s]", expand=True)
      .isin(["love"])
      .pipe(love_criteria)
     )
new_df.movie_plot_category.where(~(condition), other="love_and_death", inplace=True)
print(new_df.groupby("movie_plot_category").agg({"imdb_score": ["mean", "count", "sum"]}))

round(new_df.groupby("movie_plot_category").agg({"imdb_score": "mean"}).loc["love", "imdb_score"], 2)

                    imdb_score               
                          mean count      sum
movie_plot_category                          
death                 6.519886   176   1147.5
love                  6.588987   227   1495.7
love_and_death        6.672727    11     73.4
other                 6.431432  4629  29771.1


6.59

Колонка budget - бюджет фильма. Какой медианный бюджет всех представленных фильмов? Ответ дайте целым числом.

In [280]:
df.budget.describe()

count     5043
unique     440
top       0.0$
freq       492
Name: budget, dtype: object

In [299]:
df.budget.str.split("$", expand=True)[0].astype("float").sort_values().median()


15000000.0

## Вторая часть

Загрузите датасет event_data.csv, который содержит данные использования мобильного приложения пользователей, зарегистрировавшихся в период с 29 июля по 1 сентября 2019 года:

user_id - идентификатор пользователя;
event_date - время совершения события;
event_type - тип события: registration - регистрация в приложении; simple_event - событие клика в приложении; purchase - событие покупки внутри приложения; purchase_amount - величина покупки.
Выделите когорты пользователей на основе недели регистрации в приложении. Идентификатором когорты должен быть порядковый номер недели (например, неделя с 29 июля по 4 августа должна иметь идентификатор 31).

Какое количество уникальных пользователей в когорте с идентификатором 33?
